# Weather Classification

**Notebook:** `Weather_Classification.ipynb`

This notebook builds a supervised classification pipeline to predict the `Weather` condition using Decision Trees. **Note:** The dataset file `Weather Data.csv` is **not** included in this repo/notebook (as requested). Place `Weather Data.csv` in the same folder as this notebook before running.

---

**What this notebook contains:**
- Data loading & overview
- Exploratory data analysis
- Preprocessing (imputation + encoding)
- Decision Tree training for multiple `max_depth` values
- Train vs test accuracy table, plots, feature importances
- Save results to CSV and a short report text file


In [ ]:
# Imports
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline


In [ ]:
# Dataset path (place `Weather Data.csv` in the same folder as this notebook)
data_path = "Weather Data.csv"

if not os.path.exists(data_path):
    raise FileNotFoundError(f"Dataset not found: {data_path}.\nPlease put 'Weather Data.csv' in the same folder as this notebook and re-run.")
else:
    print("Dataset found:", data_path)

In [ ]:
# Load dataset
df = pd.read_csv(data_path)
print("Shape:", df.shape)
display(df.head(8))
display(df.dtypes)

## Exploratory Analysis
- Show class distribution for the target `Weather`
- Check missing values and constant columns


In [ ]:
# Target distribution and missing values
target_col = "Weather"
if target_col not in df.columns:
    raise ValueError(f"Expected target column named '{target_col}' not found. Columns: {df.columns.tolist()}")

print("Class distribution:")
print(df[target_col].value_counts(dropna=False))
print("\nMissing values per column:")
print(df.isna().sum().sort_values(ascending=False).head(20))

## Preprocessing
- Separate X and y
- Impute missing values (median for numeric, mode for categorical)
- Encode categorical features (one-hot)
- Train-test split (80/20). Uses `stratify` when possible.


In [ ]:
# Preprocessing
X = df.drop(columns=[target_col]).copy()
y = df[target_col].copy()

num_cols = X.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = X.select_dtypes(include=['object', 'category', 'bool']).columns.tolist()

# Impute numeric with median
for c in num_cols:
    if X[c].isna().any():
        X[c].fillna(X[c].median(), inplace=True)

# Impute categorical with mode or 'MISSING'
for c in cat_cols:
    if X[c].isna().any():
        mode = X[c].mode()
        X[c].fillna(mode.iloc[0] if not mode.empty else "MISSING", inplace=True)

# One-hot encode categorical features
X_encoded = pd.get_dummies(X, drop_first=False)

# Decide whether to stratify
min_class_count = y.value_counts().min()
strat = y if min_class_count >= 2 else None
if strat is None:
    print(f"Warning: smallest class has {min_class_count} sample(s). Proceeding without stratify.")
else:
    print("Using stratify for train_test_split.")

X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42, stratify=strat)
print("Train shape:", X_train.shape, "Test shape:", X_test.shape)

## Decision Tree training
Train DecisionTreeClassifier for `max_depth` values 1..9 and `None`. Record train and test accuracy for each.


In [ ]:
depths = list(range(1,10)) + [None]
results = []
for d in depths:
    clf = DecisionTreeClassifier(max_depth=d, random_state=42)
    clf.fit(X_train, y_train)
    train_acc = accuracy_score(y_train, clf.predict(X_train))
    test_acc = accuracy_score(y_test, clf.predict(X_test))
    results.append({'max_depth': 'None' if d is None else d, 'train_accuracy': train_acc, 'test_accuracy': test_acc})

results_df = pd.DataFrame(results)
display(results_df)

In [ ]:
# Plot train vs test accuracy
x = list(range(len(depths)))
plt.figure(figsize=(8,4))
plt.plot(x, results_df['train_accuracy'], marker='o', label='train_accuracy')
plt.plot(x, results_df['test_accuracy'], marker='o', label='test_accuracy')
plt.xticks(x, [str(d) for d in depths])
plt.xlabel('max_depth (None = unlimited)')
plt.ylabel('Accuracy')
plt.title('Train vs Test Accuracy by Decision Tree max_depth')
plt.grid(True)
plt.legend()
plt.show()

## Analysis & Conclusion
- Identify best test accuracy and discuss overfitting.
- Save results and a short report.


In [ ]:
# Best depth by test accuracy (tie-breaker: prefer smaller depth)
best_idx = results_df['test_accuracy'].idxmax()
best_row = results_df.loc[best_idx]
print("Best test accuracy:", best_row['test_accuracy'], "at max_depth=", best_row['max_depth'])

# Save results
results_df.to_csv('dt_depth_results.csv', index=False)
with open('dt_report.txt','w') as f:
    f.write(f"Best depth: {best_row['max_depth']}\nTest acc: {best_row['test_accuracy']:.4f}\nTrain acc: {best_row['train_accuracy']:.4f}\n")
print("Saved dt_depth_results.csv and dt_report.txt")

## Feature importances (final model)
Retrain final model at selected best depth and show top features.


In [ ]:
# Retrain final model and show feature importances
chosen = None if best_row['max_depth']=='None' else int(best_row['max_depth'])
final_clf = DecisionTreeClassifier(max_depth=chosen, random_state=42)
final_clf.fit(X_train, y_train)
feat_imp = pd.Series(final_clf.feature_importances_, index=X_encoded.columns).sort_values(ascending=False).head(20)
display(feat_imp.reset_index().rename(columns={'index':'feature', 0:'importance'}))

---
### Notes & next steps
- Use cross-validation (`GridSearchCV`) for robust hyperparameter selection.
- Address class imbalance (resampling or `class_weight='balanced'`) before trusting final metrics.
- Consider Random Forest / Gradient Boosting for improved generalization.

**Good luck with your submission!**
